In [ ]:
######################
#Brief:
    # 5 layer DNN ,128 neurons per layer
    # implement:
    #  batch input(for GPU optimize)
    #  N-fold cross validation
    #  print accuacy,precision,recall
    #  Dropout
    #  Optimize:
    #  batch_norm,specify momentum parameter of Adamoptimizer
#input:
    # mnist hand writing dataset 28*28 image(0~5 only)
    # output: 0~5 predicted number
#Result:
    # We found that 5 layers is unnecessary,the network can 
    # perform well (faster) with only 2 layers
#Training process:
    #input 0~5 hand writting image and it's labels from MNIST
    #image will be flatten to 1x784 array 
    #we preprocess the input with batch norm and feed them into 5 layers DNN 
    #Since ELU activation func is applied ,we use He initialize can prevent gradient from vanish
    #The softmax function will highlight the maximun output and also keep some information of other
    #which will make backward propogation faster and also ensure no weight become starvation
    #The dropout technic will prevent lazy neurons
    #We analyze accuracy,precision,recall by calculating the TP FP TN FN of each label
    #The N-fold cross validation provide us a more reliable way to analyze data
####################
import numpy as np
import os
import tensorflow as tf
from tensorflow.contrib.framework import arg_scope
from tensorflow.contrib.layers import fully_connected, batch_norm

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()


# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5] #(28038,784)
y_train1 = mnist.train.labels[mnist.train.labels < 5] #(28038,)

X_valid1 = mnist.validation.images[mnist.validation.labels < 5] #(2558,784)
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5] #(2558,) 

X_test1 = mnist.test.images[mnist.test.labels < 5] #(5139,784)
y_test1 = mnist.test.labels[mnist.test.labels < 5] #(5139,)

###### Do not modify here ###### 

#get next batch in order
def next_batch(batch_size,iteration, data, labels):
    start = batch_size*iteration
    end = batch_size*(iteration+1)
    return data[start:end], labels[start:end]

#const parameters
n_inputs = 784  # MNIST
n_outputs = 5

#adjustable parameters
N_neurons = 128
learning_rate = 0.01
momentum = 0.25
epochs = 5
batch_size = 128  #for GPU optimize
dropout = 0.5

# Create the model
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")
is_training = tf.placeholder(tf.bool, shape=(), name='is_training')


he_init = tf.contrib.layers.variance_scaling_initializer()
#improving traning speed
batch_norm_params = {
    'is_training': is_training,
    'decay': 0.9,
    'updates_collections': None,
    'scale': True,
}


#5 fully connected layer ,128 neurons per layer with dropout
with arg_scope(
        [fully_connected],
        activation_fn=tf.nn.elu,
        weights_initializer=he_init,
        normalizer_fn=batch_norm,
        normalizer_params=batch_norm_params):
    W1 = fully_connected(X,N_neurons)
    W1_D = tf.nn.dropout(W1,dropout)
    W2 = fully_connected(W1_D,N_neurons)
    W2_D = tf.nn.dropout(W2,dropout)
#     W3 = fully_connected(W2_D,N_neurons)
#     W3_D = tf.nn.dropout(W3,dropout)
#     W4 = fully_connected(W3,N_neurons)
#     W4_D = tf.nn.dropout(W4,dropout)
#     W5 = fully_connected(W4_D,N_neurons)
#     W5_D = tf.nn.dropout(W5,dropout)
    y_hat = fully_connected(W2_D, n_outputs, activation_fn=None)


#Add softmax to output

my_logits = tf.layers.dense(y_hat,n_outputs,kernel_initializer = he_init,name='logits')
# y_proba = tf.nn.softmax(logits ,name  = 'y_proba')
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=my_logits)
loss = tf.reduce_mean(cross_entropy, name="loss")

#use AdamOptimizer
optimizer = tf.train.AdamOptimizer(learning_rate, momentum)
training_op = optimizer.minimize(loss,name = "training_op")

correct = tf.nn.in_top_k(my_logits,y,1)
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32), name="accuracy")


init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    for epoch in range(epochs):
        for iteration in range(len(X_train1)//batch_size):
            X_batch, y_batch = next_batch(batch_size,iteration,X_train1, y_train1)
            sess.run(training_op, feed_dict={is_training: True, X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={is_training: False, X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={is_training: False, X:  X_valid1, y:  y_valid1})
        print(epoch, "Train accuracy:", acc_train, "Valid accuracy:", acc_valid)
    save_path = saver.save(sess, "./model/Team11_HW2.ckpt")
    acc_test = accuracy.eval(feed_dict={is_training: False, X:  X_test1, y:  y_test1})
    print( "Test accuracy:", acc_test)

In [5]:
from scipy import io
A = io.loadmat('Xia_A.mat');
print(A['a_Xn'])

[[ 0.01993286+0.j          0.01466519+0.j          0.08859343+0.j         ...,
   0.27401088+0.j          0.17266574+0.j         -0.17422963+0.j        ]
 [-0.00708883+0.00490167j  0.03841917+0.01320767j  0.08635980-0.13957259j
  ...,  0.37300627-0.0150419j   0.22107668-0.01083078j
  -0.17715369-0.01129598j]
 [ 0.04097663-0.01228058j  0.05279142+0.02322248j  0.10654337+0.24826878j
  ...,  0.30863505+0.04689259j  0.19750195-0.03083252j
  -0.16314685-0.02511039j]
 ..., 
 [-0.00736117-0.00931526j -0.02013156-0.00514053j  0.09367973+0.36155225j
  ...,  0.32212587-0.01109233j  0.22372240+0.02917594j
  -0.18690983+0.04360965j]
 [ 0.04097663+0.01228058j  0.05279142-0.02322248j  0.10654337-0.24826878j
  ...,  0.30863505-0.04689259j  0.19750195+0.03083252j
  -0.16314685+0.02511039j]
 [-0.00708883-0.00490167j  0.03841917-0.01320767j  0.08635980+0.13957259j
  ...,  0.37300627+0.0150419j   0.22107668+0.01083078j
  -0.17715369+0.01129598j]]
